In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.cross_validation import KFold, train_test_split
from tensorflow.feature_column import indicator_column
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
tf.__version__

'1.11.0'

In [3]:
df_train = pd.read_csv("training_data_post_processing2")

In [4]:
df_test = pd.read_csv("test_data_post_processing2")

In [5]:
df_train.head(3)

,first_active_month,card_id,feature_1,feature_2,feature_3,target,ave_installments,ave_month_lag,ave_purchase_amount,total_purchase_amount,total_purchase_numbers,merchant_category_id,category_1,category_2,category_3,city_id,state_id,subsector_id,month_lag
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,0.015385,-3.911538,-0.638341,-165.968739,260,"[195, 278, 683, 309, 278, 560, 560, 560, 80, 5...","['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","[69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 6...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...","[34, 37, 34, 21, 37, 34, 34, 34, 37, 34, 34, 4...","[-3, -7, -6, -2, -7, -6, -1, -7, -5, -7, 0, -2..."
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,1.560845,-5.031429,-0.600018,-210.006336,350,"[68, 528, 68, 80, 278, 278, 879, 560, 307, 560...","['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","['C', 'B', 'C', 'B', 'B', 'B', 'B', 'B', 'B', ...","[69, 69, 69, 69, 138, 69, 69, 69, 69, 69, 69, ...","[9, 9, 9, 9, 15, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,...","[27, 25, 27, 37, 37, 37, 29, 34, 19, 34, 37, 3...","[-4, -12, -11, -12, -3, -1, 0, -1, -1, 0, -5, ..."
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,0.000000,-8.604651,-0.678311,-29.167391,43,"[705, 705, 705, 705, 705, 705, 705, 705, 705, ...","['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 1.0, 5.0, ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","[143, 143, 143, 143, 143, 143, 143, 233, 143, ...","[5, 5, 5, 5, 5, 5, 5, 9, 5, 5, 5, 5, 5, 5, 9, ...","[33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 3...","[-6, -12, -9, -7, -11, -10, -4, -13, -2, -11, ..."


In [6]:
df_test.head(3)

,first_active_month,card_id,feature_1,feature_2,feature_3,ave_installments,ave_month_lag,ave_purchase_amount,total_purchase_amount,total_purchase_numbers,merchant_category_id,category_1,category_2,category_3,city_id,state_id,subsector_id,month_lag
0,2017-04,C_ID_0ab67a22ab,3,3,1,2.056338,-3.394366,-0.598745,-42.510888,71,"[412, 278, 879, 80, 278, 307, 879, 879, 80, 29...","['N', 'N', 'Y', 'N', 'N', 'N', 'N', 'Y', 'N', ...","[1.0, 1.0, nan, 1.0, 1.0, 1.0, 1.0, nan, 1.0, ...","['B', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', ...","[235, 235, -1, 235, 235, 79, 69, -1, 235, 235,...","[12, 12, -1, 12, 12, 12, 9, -1, 12, 12, -1, -1...","[18, 37, 29, 37, 37, 19, 29, 29, 37, 10, 29, 7...","[-8, -3, -6, -8, -2, 0, -5, -6, -2, -6, -8, -8..."
1,2017-01,C_ID_130fd0cbdd,2,3,0,1.080460,-9.183908,-0.633117,-55.081212,87,"[705, 705, 705, 705, 130, 87, 367, 705, 195, 1...","['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', ...","[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, ...","['B', 'B', 'B', 'B', 'B', 'C', 'B', 'B', 'B', ...","[117, 117, 117, 117, 117, 117, 117, 117, 117, ...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[33, 33, 33, 33, 41, 27, 16, 33, 34, 41, 33, 1...","[-10, -13, -10, -11, -9, -10, -9, -10, -11, -1..."
2,2017-08,C_ID_b709037bc5,5,1,1,3.996247,-1.666667,0.313932,4.708978,15,"[273, 422, 793, 769, 422, 529, 273, 87, 87, 87...","['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","['C', 'C', nan, 'C', 'C', nan, 'C', 'C', 'C', ...","[143, 143, 143, 149, 143, 143, 143, 143, 143, ...","[5, 5, 5, 20, 5, 5, 5, 5, 5, -1, 5, 9, 5, 21, -1]","[20, 27, 38, 32, 27, 20, 20, 27, 27, 29, 20, 2...","[-1, -2, -2, -4, 0, 0, -1, -2, -3, -3, 0, -6, ..."


In [8]:
scale_col = ['ave_installments', 'ave_month_lag', 'ave_purchase_amount', 'total_purchase_amount', 'total_purchase_numbers']
train_mean = np.array(df_train.loc[:, scale_col].mean()).reshape(1, -1)

In [9]:
train_mean

array([[ 0.87457267, -3.8409938 ,  2.45943043, 14.42567413, 89.29416047]])

In [10]:
train_std = np.array(df_train.loc[:, scale_col].std()).reshape(1, -1)

In [14]:
train_std[0]

array([1.02080325e+00, 2.12349449e+00, 1.11516919e+03, 1.34491579e+04,
       1.04605980e+02])

In [15]:
def df_scaling(df, train_mean, train_std, scaling_col):
    df.loc[:,scaling_col] = (df.loc[:,scaling_col] - train_mean[0])/train_std[0]
    return df   

In [16]:
df_train_scaled = df_scaling(df_train, train_mean, train_std, scale_col)
df_test_scaled = df_scaling(df_test, train_mean, train_std, scale_col)

In [17]:
df_train_scaled.head(3)

,first_active_month,card_id,feature_1,feature_2,feature_3,target,ave_installments,ave_month_lag,ave_purchase_amount,total_purchase_amount,total_purchase_numbers,merchant_category_id,category_1,category_2,category_3,city_id,state_id,subsector_id,month_lag
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,-0.841678,-0.033221,-0.002778,-0.013413,1.631894,"[195, 278, 683, 309, 278, 560, 560, 560, 80, 5...","['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","[69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 6...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...","[34, 37, 34, 21, 37, 34, 34, 34, 37, 34, 34, 4...","[-3, -7, -6, -2, -7, -6, -1, -7, -5, -7, 0, -2..."
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,0.672287,-0.560602,-0.002743,-0.016687,2.492265,"[68, 528, 68, 80, 278, 278, 879, 560, 307, 560...","['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","['C', 'B', 'C', 'B', 'B', 'B', 'B', 'B', 'B', ...","[69, 69, 69, 69, 138, 69, 69, 69, 69, 69, 69, ...","[9, 9, 9, 9, 15, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,...","[27, 25, 27, 37, 37, 37, 29, 34, 19, 34, 37, 3...","[-4, -12, -11, -12, -3, -1, 0, -1, -1, 0, -5, ..."
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,-0.856749,-2.243310,-0.002814,-0.003241,-0.442557,"[705, 705, 705, 705, 705, 705, 705, 705, 705, ...","['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 1.0, 5.0, ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","[143, 143, 143, 143, 143, 143, 143, 233, 143, ...","[5, 5, 5, 5, 5, 5, 5, 9, 5, 5, 5, 5, 5, 5, 9, ...","[33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 3...","[-6, -12, -9, -7, -11, -10, -4, -13, -2, -11, ..."


In [18]:
df_train_scaled.head(3)

,first_active_month,card_id,feature_1,feature_2,feature_3,target,ave_installments,ave_month_lag,ave_purchase_amount,total_purchase_amount,total_purchase_numbers,merchant_category_id,category_1,category_2,category_3,city_id,state_id,subsector_id,month_lag
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,-0.841678,-0.033221,-0.002778,-0.013413,1.631894,"[195, 278, 683, 309, 278, 560, 560, 560, 80, 5...","['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","[69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 6...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...","[34, 37, 34, 21, 37, 34, 34, 34, 37, 34, 34, 4...","[-3, -7, -6, -2, -7, -6, -1, -7, -5, -7, 0, -2..."
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,0.672287,-0.560602,-0.002743,-0.016687,2.492265,"[68, 528, 68, 80, 278, 278, 879, 560, 307, 560...","['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","['C', 'B', 'C', 'B', 'B', 'B', 'B', 'B', 'B', ...","[69, 69, 69, 69, 138, 69, 69, 69, 69, 69, 69, ...","[9, 9, 9, 9, 15, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,...","[27, 25, 27, 37, 37, 37, 29, 34, 19, 34, 37, 3...","[-4, -12, -11, -12, -3, -1, 0, -1, -1, 0, -5, ..."
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,-0.856749,-2.243310,-0.002814,-0.003241,-0.442557,"[705, 705, 705, 705, 705, 705, 705, 705, 705, ...","['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 1.0, 5.0, ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","[143, 143, 143, 143, 143, 143, 143, 233, 143, ...","[5, 5, 5, 5, 5, 5, 5, 9, 5, 5, 5, 5, 5, 5, 9, ...","[33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 3...","[-6, -12, -9, -7, -11, -10, -4, -13, -2, -11, ..."


In [18]:
df_test_scaled.head(2)

,first_active_month,card_id,feature_1,feature_2,feature_3,ave_installments,ave_month_lag,ave_purchase_amount,total_purchase_amount,total_purchase_numbers,merchant_category_id,category_1,category_2,category_3,city_id,state_id,subsector_id,month_lag
0,2017-04,C_ID_0ab67a22ab,3,3,1,1.157682,0.210327,-0.002742,-0.004233,-0.174886,"[412, 278, 879, 80, 278, 307, 879, 879, 80, 29...","['N', 'N', 'Y', 'N', 'N', 'N', 'N', 'Y', 'N', ...","[1.0, 1.0, nan, 1.0, 1.0, 1.0, 1.0, nan, 1.0, ...","['B', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', ...","[235, 235, -1, 235, 235, 79, 69, -1, 235, 235,...","[12, 12, -1, 12, 12, 12, 9, -1, 12, 12, -1, -1...","[18, 37, 29, 37, 37, 19, 29, 29, 37, 10, 29, 7...","[-8, -3, -6, -8, -2, 0, -5, -6, -2, -6, -8, -8..."
1,2017-01,C_ID_130fd0cbdd,2,3,0,0.201691,-2.516095,-0.002773,-0.005168,-0.021931,"[705, 705, 705, 705, 130, 87, 367, 705, 195, 1...","['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', ...","[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, ...","['B', 'B', 'B', 'B', 'B', 'C', 'B', 'B', 'B', ...","[117, 117, 117, 117, 117, 117, 117, 117, 117, ...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[33, 33, 33, 33, 41, 27, 16, 33, 34, 41, 33, 1...","[-10, -13, -10, -11, -9, -10, -9, -10, -11, -1..."


In [19]:
np.random.seed(seed=1) #makes result reproducible
msk = np.random.rand(len(df_train)) < 0.8
traindf = df_train_scaled[msk]
valdf = df_train_scaled[~msk]

In [20]:
df_train['feature_1'].unique()

array([5, 4, 2, 1, 3])

In [21]:
df_train['feature_2'].unique()

array([2, 1, 3])

In [22]:
df_train['feature_3'].unique()

array([1, 0])

In [80]:
def create_feature_cols():
    return [
    indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('feature_1', vocabulary_list=[1, 2, 3, 4, 5])),
    indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('feature_2', vocabulary_list=[1, 2, 3])),
    indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('feature_3', vocabulary_list=[0, 1])),    
    tf.feature_column.numeric_column('ave_installments', dtype=tf.float32),
    tf.feature_column.numeric_column('ave_month_lag', dtype=tf.float32),
    tf.feature_column.numeric_column('ave_purchase_amount', dtype=tf.float32),
    tf.feature_column.numeric_column('total_purchase_amount', dtype=tf.float32),
    tf.feature_column.numeric_column('total_purchase_numbers', dtype=tf.float32)]

In [81]:
input_cols = ['feature_1', 'feature_2', 'feature_3', 'ave_installments', 'ave_month_lag', 'ave_purchase_amount', 'total_purchase_amount', 'total_purchase_numbers']

In [88]:
def make_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df[input_cols],
    y = df['target'],
    batch_size = 256,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )

In [105]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = './elo_model2/'
model_dnn = tf.estimator.DNNRegressor(hidden_units=[64,64,8], model_dir=OUTDIR, feature_columns = create_feature_cols(),  optimizer=lambda: tf.train.FtrlOptimizer(
        learning_rate=tf.train.exponential_decay(
            learning_rate=0.1,
            global_step=tf.train.get_global_step(),
            decay_steps=10000,
            decay_rate=0.96)))
#   train_spec = tf.estimator.TrainSpec(input_fn = make_input_fn(traindf, None), 
#                                   max_steps = num_train_steps)
#   eval_spec = tf.estimator.EvalSpec(input_fn = make_input_fn(valdf, 1), 
#                                     steps = None, 
#                                     start_delay_secs = 1, # start evaluating after N seconds, 
#                                     throttle_secs = 5)  # evaluate every N seconds
model_dnn.train(input_fn = make_input_fn(traindf, None), steps = 100000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './elo_model2/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f842b37c278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was f

INFO:tensorflow:loss = 6838.29, step = 7200 (0.769 sec)
INFO:tensorflow:global_step/sec: 132.077
INFO:tensorflow:loss = 5722.931, step = 7300 (0.757 sec)
INFO:tensorflow:global_step/sec: 129.137
INFO:tensorflow:loss = 7937.385, step = 7400 (0.774 sec)
INFO:tensorflow:global_step/sec: 128.036
INFO:tensorflow:loss = 7943.2866, step = 7500 (0.781 sec)
INFO:tensorflow:global_step/sec: 128.295
INFO:tensorflow:loss = 8155.7393, step = 7600 (0.781 sec)
INFO:tensorflow:global_step/sec: 129.387
INFO:tensorflow:loss = 3761.8384, step = 7700 (0.771 sec)
INFO:tensorflow:global_step/sec: 130.304
INFO:tensorflow:loss = 9253.871, step = 7800 (0.767 sec)
INFO:tensorflow:global_step/sec: 130.852
INFO:tensorflow:loss = 7671.1953, step = 7900 (0.764 sec)
INFO:tensorflow:global_step/sec: 128.251
INFO:tensorflow:loss = 6157.4956, step = 8000 (0.780 sec)
INFO:tensorflow:global_step/sec: 130.097
INFO:tensorflow:loss = 11147.12, step = 8100 (0.769 sec)
INFO:tensorflow:global_step/sec: 130.91
INFO:tensorflow:l

INFO:tensorflow:loss = 8901.43, step = 15500 (0.780 sec)
INFO:tensorflow:global_step/sec: 130.73
INFO:tensorflow:loss = 9122.084, step = 15600 (0.771 sec)
INFO:tensorflow:global_step/sec: 128.585
INFO:tensorflow:loss = 5911.528, step = 15700 (0.772 sec)
INFO:tensorflow:global_step/sec: 128.475
INFO:tensorflow:loss = 7792.4253, step = 15800 (0.778 sec)
INFO:tensorflow:global_step/sec: 130.741
INFO:tensorflow:loss = 3461.5054, step = 15900 (0.765 sec)
INFO:tensorflow:global_step/sec: 128.334
INFO:tensorflow:loss = 3844.9985, step = 16000 (0.779 sec)
INFO:tensorflow:global_step/sec: 131.231
INFO:tensorflow:loss = 7266.9316, step = 16100 (0.762 sec)
INFO:tensorflow:global_step/sec: 130.267
INFO:tensorflow:loss = 6337.3345, step = 16200 (0.768 sec)
INFO:tensorflow:global_step/sec: 129.645
INFO:tensorflow:loss = 4482.2036, step = 16300 (0.772 sec)
INFO:tensorflow:global_step/sec: 129.637
INFO:tensorflow:loss = 6027.9453, step = 16400 (0.771 sec)
INFO:tensorflow:global_step/sec: 129.876
INFO:

INFO:tensorflow:global_step/sec: 132.034
INFO:tensorflow:loss = 7920.3545, step = 23800 (0.757 sec)
INFO:tensorflow:global_step/sec: 128.48
INFO:tensorflow:loss = 12193.098, step = 23900 (0.778 sec)
INFO:tensorflow:global_step/sec: 128.486
INFO:tensorflow:loss = 7915.796, step = 24000 (0.779 sec)
INFO:tensorflow:global_step/sec: 131.611
INFO:tensorflow:loss = 4932.5947, step = 24100 (0.760 sec)
INFO:tensorflow:global_step/sec: 129.309
INFO:tensorflow:loss = 9345.375, step = 24200 (0.773 sec)
INFO:tensorflow:global_step/sec: 131.659
INFO:tensorflow:loss = 6856.0425, step = 24300 (0.760 sec)
INFO:tensorflow:global_step/sec: 128.633
INFO:tensorflow:loss = 12144.985, step = 24400 (0.777 sec)
INFO:tensorflow:global_step/sec: 130.72
INFO:tensorflow:loss = 12511.084, step = 24500 (0.765 sec)
INFO:tensorflow:global_step/sec: 127.349
INFO:tensorflow:loss = 9964.274, step = 24600 (0.785 sec)
INFO:tensorflow:global_step/sec: 131.379
INFO:tensorflow:loss = 5798.257, step = 24700 (0.761 sec)
INFO:t

INFO:tensorflow:global_step/sec: 130.486
INFO:tensorflow:loss = 4803.4023, step = 32100 (0.767 sec)
INFO:tensorflow:global_step/sec: 131.543
INFO:tensorflow:loss = 10113.303, step = 32200 (0.760 sec)
INFO:tensorflow:global_step/sec: 128.857
INFO:tensorflow:loss = 7825.4404, step = 32300 (0.776 sec)
INFO:tensorflow:global_step/sec: 129.986
INFO:tensorflow:loss = 5969.204, step = 32400 (0.769 sec)
INFO:tensorflow:global_step/sec: 129.901
INFO:tensorflow:loss = 9222.123, step = 32500 (0.770 sec)
INFO:tensorflow:global_step/sec: 129.164
INFO:tensorflow:loss = 9094.364, step = 32600 (0.774 sec)
INFO:tensorflow:global_step/sec: 130.854
INFO:tensorflow:loss = 4719.965, step = 32700 (0.764 sec)
INFO:tensorflow:global_step/sec: 128.264
INFO:tensorflow:loss = 11223.916, step = 32800 (0.780 sec)
INFO:tensorflow:global_step/sec: 130.466
INFO:tensorflow:loss = 8030.081, step = 32900 (0.767 sec)
INFO:tensorflow:global_step/sec: 128.103
INFO:tensorflow:loss = 7857.6455, step = 33000 (0.780 sec)
INFO:

...

INFO:tensorflow:global_step/sec: 128.662
INFO:tensorflow:loss = 12430.408, step = 65300 (0.777 sec)
INFO:tensorflow:global_step/sec: 127.151
INFO:tensorflow:loss = 6970.274, step = 65400 (0.786 sec)
INFO:tensorflow:global_step/sec: 128.366
INFO:tensorflow:loss = 7888.8193, step = 65500 (0.779 sec)
INFO:tensorflow:global_step/sec: 129.459
INFO:tensorflow:loss = 6846.165, step = 65600 (0.773 sec)
INFO:tensorflow:global_step/sec: 131.061
INFO:tensorflow:loss = 3549.9612, step = 65700 (0.763 sec)
INFO:tensorflow:global_step/sec: 130.791
INFO:tensorflow:loss = 5681.3594, step = 65800 (0.764 sec)
INFO:tensorflow:global_step/sec: 130.98
INFO:tensorflow:loss = 7550.679, step = 65900 (0.763 sec)
INFO:tensorflow:global_step/sec: 132.017
INFO:tensorflow:loss = 9788.372, step = 66000 (0.757 sec)
INFO:tensorflow:global_step/sec: 130.874
INFO:tensorflow:loss = 6998.07, step = 66100 (0.764 sec)
INFO:tensorflow:global_step/sec: 130.828
INFO:tensorflow:loss = 12022.6875, step = 66200 (0.764 sec)
INFO:t

INFO:tensorflow:global_step/sec: 131.808
INFO:tensorflow:loss = 13330.639, step = 73600 (0.759 sec)
INFO:tensorflow:global_step/sec: 131.741
INFO:tensorflow:loss = 3821.5005, step = 73700 (0.759 sec)
INFO:tensorflow:global_step/sec: 130.216
INFO:tensorflow:loss = 6033.612, step = 73800 (0.769 sec)
INFO:tensorflow:global_step/sec: 129.098
INFO:tensorflow:loss = 4762.6953, step = 73900 (0.776 sec)
INFO:tensorflow:global_step/sec: 132.064
INFO:tensorflow:loss = 7176.908, step = 74000 (0.755 sec)
INFO:tensorflow:global_step/sec: 133.047
INFO:tensorflow:loss = 6749.368, step = 74100 (0.752 sec)
INFO:tensorflow:global_step/sec: 130.12
INFO:tensorflow:loss = 10686.253, step = 74200 (0.769 sec)
INFO:tensorflow:global_step/sec: 128.496
INFO:tensorflow:loss = 7928.676, step = 74300 (0.778 sec)
INFO:tensorflow:global_step/sec: 128.097
INFO:tensorflow:loss = 2665.4868, step = 74400 (0.781 sec)
INFO:tensorflow:global_step/sec: 129.476
INFO:tensorflow:loss = 9983.252, step = 74500 (0.773 sec)
INFO:t

INFO:tensorflow:global_step/sec: 131.964
INFO:tensorflow:loss = 4863.5522, step = 81800 (0.758 sec)
INFO:tensorflow:global_step/sec: 133.771
INFO:tensorflow:loss = 9205.656, step = 81900 (0.748 sec)
INFO:tensorflow:global_step/sec: 132.098
INFO:tensorflow:loss = 5605.633, step = 82000 (0.756 sec)
INFO:tensorflow:global_step/sec: 131.557
INFO:tensorflow:loss = 8047.4297, step = 82100 (0.760 sec)
INFO:tensorflow:global_step/sec: 130.715
INFO:tensorflow:loss = 6086.074, step = 82200 (0.765 sec)
INFO:tensorflow:global_step/sec: 129.692
INFO:tensorflow:loss = 11846.043, step = 82300 (0.771 sec)
INFO:tensorflow:global_step/sec: 131.59
INFO:tensorflow:loss = 6410.4023, step = 82400 (0.760 sec)
INFO:tensorflow:global_step/sec: 130.031
INFO:tensorflow:loss = 6994.6, step = 82500 (0.769 sec)
INFO:tensorflow:global_step/sec: 130.741
INFO:tensorflow:loss = 5620.179, step = 82600 (0.765 sec)
INFO:tensorflow:global_step/sec: 129.249
INFO:tensorflow:loss = 10286.071, step = 82700 (0.774 sec)
INFO:ten

INFO:tensorflow:global_step/sec: 131.539
INFO:tensorflow:loss = 5118.286, step = 90100 (0.760 sec)
INFO:tensorflow:global_step/sec: 131.073
INFO:tensorflow:loss = 6776.582, step = 90200 (0.763 sec)
INFO:tensorflow:global_step/sec: 131.849
INFO:tensorflow:loss = 9961.203, step = 90300 (0.759 sec)
INFO:tensorflow:global_step/sec: 130.652
INFO:tensorflow:loss = 11050.7295, step = 90400 (0.765 sec)
INFO:tensorflow:global_step/sec: 129.811
INFO:tensorflow:loss = 8779.455, step = 90500 (0.770 sec)
INFO:tensorflow:global_step/sec: 130.045
INFO:tensorflow:loss = 6916.9424, step = 90600 (0.769 sec)
INFO:tensorflow:global_step/sec: 133.835
INFO:tensorflow:loss = 9570.806, step = 90700 (0.747 sec)
INFO:tensorflow:global_step/sec: 133.476
INFO:tensorflow:loss = 4776.7056, step = 90800 (0.749 sec)
INFO:tensorflow:global_step/sec: 129.412
INFO:tensorflow:loss = 6512.0015, step = 90900 (0.773 sec)
INFO:tensorflow:global_step/sec: 129.607
INFO:tensorflow:loss = 8991.967, step = 91000 (0.771 sec)
INFO:

INFO:tensorflow:global_step/sec: 128.106
INFO:tensorflow:loss = 12280.024, step = 98400 (0.781 sec)
INFO:tensorflow:global_step/sec: 130.099
INFO:tensorflow:loss = 12505.01, step = 98500 (0.769 sec)
INFO:tensorflow:global_step/sec: 129.948
INFO:tensorflow:loss = 3747.6802, step = 98600 (0.770 sec)
INFO:tensorflow:global_step/sec: 128.328
INFO:tensorflow:loss = 8466.977, step = 98700 (0.779 sec)
INFO:tensorflow:global_step/sec: 129.674
INFO:tensorflow:loss = 12485.292, step = 98800 (0.771 sec)
INFO:tensorflow:global_step/sec: 130.167
INFO:tensorflow:loss = 9362.296, step = 98900 (0.768 sec)
INFO:tensorflow:global_step/sec: 130.259
INFO:tensorflow:loss = 6505.827, step = 99000 (0.768 sec)
INFO:tensorflow:global_step/sec: 129.823
INFO:tensorflow:loss = 9223.947, step = 99100 (0.770 sec)
INFO:tensorflow:global_step/sec: 129.615
INFO:tensorflow:loss = 5778.033, step = 99200 (0.772 sec)
INFO:tensorflow:global_step/sec: 130.119
INFO:tensorflow:loss = 5515.8735, step = 99300 (0.769 sec)
INFO:t

In [106]:
def rmse(model):
  metrics = model.evaluate(input_fn = make_input_fn(valdf, 1))
  return np.sqrt(metrics['average_loss'])

In [107]:
print(rmse(model_dnn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-29-05:06:42
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./elo_model2/model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-29-05:06:43
INFO:tensorflow:Saving dict for global step 100000: average_loss = 14.942004, global_step = 100000, label/mean = -0.41415998, loss = 7620.9897, prediction/mean = -0.39414275
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100000: ./elo_model2/model.ckpt-100000
3.8654888


In [108]:
submit = pd.read_csv("./input/sample_submission.csv")

In [109]:
submit.head()

,card_id,target
0,C_ID_0ab67a22ab,0
1,C_ID_130fd0cbdd,0
2,C_ID_b709037bc5,0
3,C_ID_d27d835a9f,0
4,C_ID_2b5e3df5c2,0


In [111]:
def make_input_fn_pred(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df[input_cols],
    batch_size = 256,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )

In [115]:
pred = list(model_dnn.predict(make_input_fn_pred(df_test_scaled, 1)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./elo_model2/model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [123]:
for idx, val in enumerate(pred):
    submit.loc[idx, "target"] = val['predictions']

In [125]:
submit.tail()

,card_id,target
123618,C_ID_7a239d2eda,-0.367746
123619,C_ID_75ace375ae,-1.057627
123620,C_ID_21d56d950c,-0.162090
123621,C_ID_6c46fc5a9d,-0.162090
123622,C_ID_87e7979a5f,-0.641419


In [127]:
submit.to_csv("submission1.csv", index=False)

In [129]:
pd.read_csv("submission1.csv").tail()

,card_id,target
123618,C_ID_7a239d2eda,-0.367746
123619,C_ID_75ace375ae,-1.057627
123620,C_ID_21d56d950c,-0.162090
123621,C_ID_6c46fc5a9d,-0.162090
123622,C_ID_87e7979a5f,-0.641419


In [132]:
df_test_scaled2 = df_test_scaled.copy()

In [134]:
df_test_scaled2.head(2)

,first_active_month,card_id,feature_1,feature_2,feature_3,ave_installments,ave_month_lag,ave_purchase_amount,total_purchase_amount,total_purchase_numbers,merchant_category_id,category_1,category_2,category_3,city_id,state_id,subsector_id,month_lag
0,2017-04,C_ID_0ab67a22ab,3,3,1,1.157682,0.210327,-0.002742,-0.004233,-0.174886,"[412, 278, 879, 80, 278, 307, 879, 879, 80, 29...","['N', 'N', 'Y', 'N', 'N', 'N', 'N', 'Y', 'N', ...","[1.0, 1.0, nan, 1.0, 1.0, 1.0, 1.0, nan, 1.0, ...","['B', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', ...","[235, 235, -1, 235, 235, 79, 69, -1, 235, 235,...","[12, 12, -1, 12, 12, 12, 9, -1, 12, 12, -1, -1...","[18, 37, 29, 37, 37, 19, 29, 29, 37, 10, 29, 7...","[-8, -3, -6, -8, -2, 0, -5, -6, -2, -6, -8, -8..."
1,2017-01,C_ID_130fd0cbdd,2,3,0,0.201691,-2.516095,-0.002773,-0.005168,-0.021931,"[705, 705, 705, 705, 130, 87, 367, 705, 195, 1...","['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', ...","[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, ...","['B', 'B', 'B', 'B', 'B', 'C', 'B', 'B', 'B', ...","[117, 117, 117, 117, 117, 117, 117, 117, 117, ...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[33, 33, 33, 33, 41, 27, 16, 33, 34, 41, 33, 1...","[-10, -13, -10, -11, -9, -10, -9, -10, -11, -1..."


In [137]:
df_test_scaled2['ave_month_lag'] = df_test_scaled2['ave_month_lag'] -2
# The evalution time is 2 month later than the training data with new transaction data merged in.  Time was adjusted by -2.  

In [138]:
df_test_scaled2.head(2)

,first_active_month,card_id,feature_1,feature_2,feature_3,ave_installments,ave_month_lag,ave_purchase_amount,total_purchase_amount,total_purchase_numbers,merchant_category_id,category_1,category_2,category_3,city_id,state_id,subsector_id,month_lag
0,2017-04,C_ID_0ab67a22ab,3,3,1,1.157682,-1.789673,-0.002742,-0.004233,-0.174886,"[412, 278, 879, 80, 278, 307, 879, 879, 80, 29...","['N', 'N', 'Y', 'N', 'N', 'N', 'N', 'Y', 'N', ...","[1.0, 1.0, nan, 1.0, 1.0, 1.0, 1.0, nan, 1.0, ...","['B', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', ...","[235, 235, -1, 235, 235, 79, 69, -1, 235, 235,...","[12, 12, -1, 12, 12, 12, 9, -1, 12, 12, -1, -1...","[18, 37, 29, 37, 37, 19, 29, 29, 37, 10, 29, 7...","[-8, -3, -6, -8, -2, 0, -5, -6, -2, -6, -8, -8..."
1,2017-01,C_ID_130fd0cbdd,2,3,0,0.201691,-4.516095,-0.002773,-0.005168,-0.021931,"[705, 705, 705, 705, 130, 87, 367, 705, 195, 1...","['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', ...","[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, ...","['B', 'B', 'B', 'B', 'B', 'C', 'B', 'B', 'B', ...","[117, 117, 117, 117, 117, 117, 117, 117, 117, ...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[33, 33, 33, 33, 41, 27, 16, 33, 34, 41, 33, 1...","[-10, -13, -10, -11, -9, -10, -9, -10, -11, -1..."


In [139]:
pred = list(model_dnn.predict(make_input_fn_pred(df_test_scaled2, 1)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./elo_model2/model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [140]:
submit2 = pd.read_csv("./input/sample_submission.csv")

In [142]:
for idx, val in enumerate(pred):
    submit2.loc[idx, "target"] = val['predictions']

In [143]:
submit2.head()

,card_id,target
0,C_ID_0ab67a22ab,-0.373222
1,C_ID_130fd0cbdd,-0.162090
2,C_ID_b709037bc5,-0.162090
3,C_ID_d27d835a9f,-0.162090
4,C_ID_2b5e3df5c2,-0.630401


In [144]:
submit2.to_csv("submission2.csv", index=False)